<h1 align="center" style="background-color:#616161;color:white">RNN Model with logloss</h1>

<h3 style="background-color:#616161;color:white">0. Setup</h3>

<div style="background-color:white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Input Parameters</div>

In [1]:
# Root path
#root = "C:/DS/Github/MusicRecommendation"  # BA, Windows
root = "/home/badrul/git/EventPrediction" # BA, Linux

<div style="background-color:white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Common Libraries</div>

In [2]:
# Core
import numpy as np
import pandas as pd
from IPython.core.debugger import Tracer    # Used for debugging
import logging
from random import *

# File and database management
import csv
import os
import sys
import json
import sqlite3
from pathlib import Path

# Date/Time
import datetime
import time
#from datetime import timedelta # Deprecated

# Visualization
import matplotlib.pyplot as plt             # Quick
%matplotlib inline

# Misc
import random
import importlib
import warnings
warnings.filterwarnings('ignore')
import logging
logging.basicConfig(filename='RNN.log',level=logging.DEBUG)

#-------------- Custom Libs -----------------#
os.chdir(root)

# Import the codebase module
fPath = root + "/1_Codemodule"
if fPath not in sys.path: sys.path.append(fPath)

# Custom Libs
import coreCode as cc
import lastfmCode as fm
print ('Ok')

Ok


<div style="background-color:white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Page Specific Libraries</div>

In [3]:
# Data science (comment out if not needed)
#from sklearn.manifold import TSNE
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.python.framework import ops
ops.reset_default_graph()
from sklearn import metrics
from sklearn import preprocessing
print ('Ok')

Ok


<div style="background-color:#white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Load settings</div>

In [4]:
settingsDict =  cc.loadSettings()
dbPath = root + settingsDict['mainDbPath_sml']
fmSimilarDbPath = root + settingsDict['fmSimilarDbPath']
fmTagsDbPath = root + settingsDict['fmTagsDbPath']
trackMetaDbPath = root + settingsDict['trackmetadata']
periodGranularity = int(settingsDict['periodGranularity'])
print ('Ok')

Ok


<h3 style="background-color:#616161;color:white">1. Build Model</h3>

In [5]:
def RNN(x, weights, biases,n_steps):
    # Current data input shape: (batchRows, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batchRows, n_input)
    
    # Unstack to get a list of 'n_steps' tensors of shape (batchRows, n_input)
    x = tf.unstack(x, n_steps, 1)  # See https://stackoverflow.com/questions/45278276/tensorflow-lstm-dropout-implementation-shape-problems/45279243#45279243
    
    # Define a lstm cell with tensorflow
    if cellType == "LSTMCell":
        stacked_rnn=[]
        for i in range(n_layers):
            stacked_rnn.append(tf.nn.rnn_cell.LSTMCell(num_units=n_hidden, forget_bias=1., state_is_tuple=True))
        multiRNNCell = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_rnn, state_is_tuple=True)
        outputs, states = rnn.static_rnn(multiRNNCell, x, dtype=tf.float32)
        
                
        #lstm_cell = rnn.BasicLSTMCell(num_units=n_hidden, forget_bias=1.0)
        #outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    
    elif cellType == "TimeFreqLSTMCell":
        lstm_cell =rnn.TimeFreqLSTMCell(n_hidden, use_peepholes=True, feature_size= 22, forget_bias=1.0)
        outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    elif cellType == "GridLSTMCell":
        lstm_cell =rnn.GridLSTMCell(n_hidden, forget_bias=1.0)
        outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)        
    else:
        print("Did not recognize {}".format(cellType))
    # Get lstm cell output
    

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [6]:
def _buildGraph(n_steps,n_input, n_weighting):
    global x, y, _pred, _predProb, _logloss, _logits, _prob, _weights, _cost, optimizer, _accuracy,_correct_pred
    
    tf.reset_default_graph()
    # tf Graph input
    x = tf.placeholder("float", [None, n_steps, n_input])
    y = tf.placeholder("float", [None,n_classes])
    
    # Define weights
    weights = {'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))}
    biases = {'out': tf.Variable(tf.random_normal([n_classes]))}

    # Evaluate model
    _logits = RNN(x, weights, biases,n_steps)
    _prob = tf.sigmoid(_logits)
    
    # Loss function
    _weights = tf.add(1,tf.multiply(tf.cast(tf.equal(y,1),'int32'),n_weighting))
    _logloss =tf.losses.log_loss(predictions=_prob, labels=y,epsilon=0.00001, weights=_weights)
    _cost = tf.reduce_mean(_logloss)
    
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(_cost)
    
    _predProb =tf.sigmoid(_logits)  # Convert to proper probs
    _pred =tf.round(_predProb)  # Take the highest prob
    _correct_pred = tf.equal(_pred, y)
    _accuracy = tf.reduce_mean(tf.cast(_correct_pred, tf.float32))
print('Ok')

Ok


In [7]:
def ResetModel():
    try:
        sess.close()
        tf.reset_default_graph()
    except NameError:
        return
    
def initializeModel(n_steps,n_input,n_weighting,loadFromSave):
    global sess

    # Build graph
    _buildGraph(n_steps,n_input,n_weighting)

    # Initializing the variables

    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    sess = tf.Session() # Has tome come after init
    if loadFromSave:
        saver.restore(sess,'./3_Data/saves/model.ckpt')
    else:
        sess.run(init)
    print('Model initialized')

<h3 style="background-color:#616161;color:white">2. Model Training Functions</h3>

In [8]:
def randomSelectFromData(_X, _Y,_batchRows = 10, numOfSamples =1):
    
    # Num of periods = batch size
    
    # Training cycle
    
    XCols=np.shape(_X)[1]
    YCols=np.shape(_Y)[1]
    depth = n_steps
    
    totalRows=np.shape(_X)[0]
    # Select random periods (ones where we will always get enough history to go with it)
    periodsList = random.sample(range(batchRows+depth, totalRows), _batchRows)
    
    # Debugging...
    #for i in range(_batchRows -1):
    #    periodsList[i+1]=periodsList[i]-1
    
    # Pre-Initialize batch arrays
    batch_x=np.zeros([_batchRows,depth,XCols])
    batch_y=np.zeros([_batchRows])

    batch_row =0
    
    for periodPos in periodsList:            
        # Log every so often 
        if (periodPos % 1) == 0: 
            timeNow =str(datetime.datetime.now())
            #print("{} Now adding random period {} into batch_row {}. ({}%)".format(timeNow,idx1,batch_row, round((batch_row/_batchRows)*100,2)))
            #logging.info("{} Now adding random period {} into batch_row ({}%)".format(timeNow, idx1,batch_row, round((batch_row/_batchRows)*100,2)))

        batch_x[batch_row] = _X[periodPos-depth:periodPos].reshape(1,depth,XCols)
        batch_y[batch_row] = _Y[periodPos]
        batch_row +=1
        
    return batch_x, batch_y
print ('Ok')

Ok


In [9]:
##############################################
###           MODEL 1    Train             ###
##############################################
def trainModel_1(batch_x, batch_y): 
    
    batch_y = np.reshape(batch_y,[-1,1])
    sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
    cost = sess.run(_cost,feed_dict={x: batch_x, y: batch_y})
    acc = sess.run(_accuracy, feed_dict={x: batch_x, y: batch_y})
    
    debugMode = False
    
    if debugMode:
        # Calculate loss & accuracy
        logloss= sess.run(_logloss,feed_dict={x: batch_x, y: batch_y})
        prob= sess.run(_prob,feed_dict={x: batch_x, y: batch_y})

        weights= sess.run(_weights, feed_dict={x: batch_x, y: batch_y})
        logits= sess.run(_logits, feed_dict={x: batch_x, y: batch_y})
        #weights= sess.run(lossW, feed_dict={x: batch_x, y: batch_y})
        #crosent= sess.run(tmp, feed_dict={x: batch_x, y: batch_y})
        print ("Logits:")
        print (logits)
        print("Prob")
        print(prob)
        print ("Y")
        print (batch_y)
        print ("LossWeight")
        print (weights)
        #print ("CrossEnt")
        #print (crosent)
        print ("Log Loss")
        print (logloss)
        print ("\nCost: {}".format(cost))
        Tracer()()
    return cost, acc
print('Ok')

Ok


<h3 style="background-color:#616161;color:white">3. Model Testing Functions</h3>

In [10]:
# Launch the graph
def TestPredictions1(_X, _Y, sess, _batchRows = 10, testPeriods = None):
    # Training cycle
    totalRows=np.shape(_X)[0]
    XCols=np.shape(_X)[1]
    YCols=_Y
    depth = n_steps
    
    # If no test periods were provided generate your own
    if testPeriods is None:
        # Select periods where we will always get enough history to go with it
        
        testPeriods = random.sample(range(batchRows+depth, totalRows), _batchRows)
    else:
        testPeriods = testPeriods + batchRows+depth-1
        _batchRows = len(testPeriods)
     
    # Pre-Initialize batch arrays
    batch_x=np.zeros([_batchRows,depth,XCols])
    batch_y=np.zeros([_batchRows])

    batch_row =0
    for idx1 in testPeriods:            
        if (idx1 % 1) == 0: 
            timeNow =str(datetime.datetime.now())
            #print("{} Now testing on period {} ({}%)".format(timeNow,idx1,round((batch_row/_batchRows)*100,2)))
            logging.info("{} Now testing period {} ({}%)".format(timeNow, idx1,round((batch_row/_batchRows)*100,2)))

        batch_x[batch_row] = _X[idx1-depth:idx1].reshape(1,depth,XCols)
        batch_y[batch_row] = _Y[idx1]
        batch_row +=1
    
    print ("Processed {}".format(_batchRows))
    # Predict for this period
    batch_y=np.reshape(batch_y,[-1,1])
    prob = sess.run(_predProb, feed_dict={x: batch_x, y: batch_y})
    predictions = sess.run(_pred, feed_dict={x: batch_x, y: batch_y})
    
    return predictions, batch_y



<div style="background-color:#white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Test hidden periods</div>

In [11]:
def TestHiddenPeriods(hiddenTestPeriods=50, useTestData = False):

    print('{} Hidden Periods\n'.format(hiddenTestPeriods))
    print ("Cell type= {}, learning_rate = {}, Iterations = {}, batch size = {}, Steps = {}, Hidden Layers = {}, Classes = {}\n".format(cellType,learning_rate,samplesPerUser,batch_size, n_steps ,n_hidden,n_classes))

    if useTestData == False:
        predictions,labels = TestPredictions1(xTrain,yTrain,sess,numOfPeriods=hiddenTestPeriods)
    else:
        predictions,labels = TestPredictions1(xTest,yTest,sess,numOfPeriods=hiddenTestPeriods)
    
    print(np.shape(labels),np.shape(predictions))    
    print(metrics.classification_report(labels,predictions))  # Need to feed it yTest not yTest_OneHot here
    return labels, predictions

In [24]:
def TrainModel(printOnBatchIteration=True,printOnSampleIteration=1,enableLogging=False, _saveModel=False):
    
    counter =0 # Used to keep track of every iteration. (Does not loop back to 0).
    batch_y=[]
    batch_x=[]
    
    loss=np.zeros([user_iteration*samplesPerUser*batch_Iterations+3])
    acc=np.zeros([user_iteration*samplesPerUser*batch_Iterations+3])
    
    
    trainUsers=(cc.getUsers(dbPath,testUserFlag=0))  # Get list of users
    np.random.shuffle(trainUsers)

    for userCount in range(user_iteration):  # Iterate through user selection
        # Get user data
        userID = int(trainUsers[userCount]) # Randomly select 1 user
        xTrain, yTrain, xTest, yTest = cc.SelectUserData_TrainTest(dbPath,tblName,fieldList,userIDs=[userID],oneHot=False,periodGranularity=periodGranularity)
        #yTrain=(xTrain[:,0]==0)*(xTrain[:,1]==0)*(xTrain[:,2]==0)*(xTrain[:,3]==0)*yTrain
        #yTest=(xTest[:,0]==0)*(xTest[:,1]==0)*(xTest[:,2]==0)*(xTest[:,3]==0)*yTest
        
        if xTrain is not None: # Make sure we have data
            for i in range(samplesPerUser):  # Num of mini-batches
                
                # Randomly select from the train data
                batch_x, batch_y = randomSelectFromData(xTrain, yTrain, _batchRows=batchRows)
                
                #print ('Num of play events in batch: {}'.format(sum(batch_y)))
                
                for j in range(batch_Iterations): # Num of timetrainModel_1s to iterate over the batch
                    loss[counter],acc[counter] =trainModel_1(batch_x,batch_y)
                    
                    if printOnBatchIteration:
                        timeNow =datetime.datetime.now().strftime('%D %H:%M:%S')
                        print("  User {} Mini-batch {} Iteration {} Loss={:.6f}, Training Accuracy={:.5f}".format(userCount,i, j, loss[counter], acc[counter]))
                        if enableLogging: logging.info(s)
                    counter+=1

                
                if i % printOnSampleIteration == 0 or i==samplesPerUser:
                    # Sample results
                    batch_y = np.reshape(batch_y,[-1,1])
                    predictions = sess.run(_pred, feed_dict={x: batch_x, y: batch_y})
                    logits= sess.run(_logits, feed_dict={x: batch_x, y: batch_y})
                    prob= sess.run(_predProb, feed_dict={x: batch_x, y: batch_y})                    
                    
                    prec,rec, _, _ = metrics.precision_recall_fscore_support(batch_y,predictions, average='binary')
                    timeNow =datetime.datetime.now().strftime('%D %H:%M:%S')
                    
                    s='{} Sample {} of {}. Precision {} .Recall {} .Loss {:.{prec}f} .Accuracy {:.{prec}f}'.format(timeNow, i, samplesPerUser,np.round(prec,3),np.round(rec,3),loss[counter-1],acc[counter-1],prec=4)
                    print(s)
                    if enableLogging: logging.info(s)
                    
            # End of user training - Perform test
            xTest, yTest = randomSelectFromData(xTest, yTest, _batchRows=batchRows, numOfSamples=2)
            yTest = np.reshape(yTest,[-1,1])
            predictions = sess.run(_pred, feed_dict={x: xTest, y: yTest})
            logits= sess.run(_logits, feed_dict={x: xTest, y: yTest})
            prob= sess.run(_predProb, feed_dict={x: xTest, y: yTest})                    

            prec,rec, _, _ = metrics.precision_recall_fscore_support(yTest,predictions, average='binary')
            
            timeNow =datetime.datetime.now().strftime('%D %H:%M:%S')
            s='{} User {} of {}, UserID {}, Total plays {}, Precision {}, Recall {}'.format(timeNow, userCount,user_iteration,userID,sum(yTrain),np.round(prec,3),np.round(rec,3))
            print(s)
            if enableLogging: logging.info(s)
            print(metrics.classification_report(yTest,predictions))  # Need to feed it yTest not yTest_OneHot here
            
            if _saveModel:
                saver = tf.train.Saver()
                saver.save(sess,"./3_Data/saves/model.ckpt")

In [17]:
### Code needs tidying up

def TestModel(_samples):
    userLabels=[]
    userPred=[]
    totalLabels = []
    totalPred = []
    avLoss =0
    avAcc =0
    
    
    users=cc.getUsers(dbPath,testUserFlag = 1)  # Get all test users

    prec= np.zeros(len(users))
    rec = np.zeros(len(users))
    
    usrCount = 0
    for usr in users: # For each test user    
        # Select all data for each user
        xTest, yTest = cc.SelectTestUserData(dbPath,tblName,fieldList,userIDs=usr,oneHot=False,periodGranularity=periodGranularity)
        #yTest=(xTest[:,0]==0)*(xTest[:,1]==0)*(x[Test:,2]==0)*(xTest[:,3]==0)*y  # Only keep first plays
        if xTest is not None:
            for i in range(_samples):  # Select 10 random batches
                batch_x, batch_y = randomSelectFromData(xTest, yTest, _batchRows=batchRows)
                batch_y = np.reshape(batch_y,[-1,1])
                p = sess.run(_pred, feed_dict={x: batch_x, y: batch_y})
                
                loss = sess.run(_cost,feed_dict={x: batch_x, y: batch_y})
                acc = sess.run(_accuracy, feed_dict={x: batch_x, y: batch_y})
                avLoss+=loss
                avAcc+=acc

                if userLabels == []:
                    userLabels= batch_y
                    userPred = p
                else:
                    userLabels = np.append(userLabels,batch_y)
                    userPred = np.append(userPred,p)
        
        prec[usrCount],rec[usrCount], _, _ = metrics.precision_recall_fscore_support(userLabels,userPred, average='binary')
        #print('User {} of {}, Av loss {}, Av acc {}, Precision {}, Recall {}'.format(usrCount , len(users), np.round(avLoss/10,3), np.round(avAcc/10,3), np.round(prec[usrCount],3),np.round(rec[usrCount],3)))
        avLoss=0
        avAcc = 0
        if totalLabels ==[]:
            totalLabels = userLabels
            totalPred = userPred
        else:
            totalLabels = np.append(totalLabels,userLabels)
            totalPred = np.append(totalPred,userPred)
        usrCount+=1                
    
    pMn=round(prec.mean(),3)
    pSd=round(prec.std(),3)
    rMn=round(rec.mean(),3)
    rSd=round(rec.std(),3)
    print(metrics.classification_report(totalLabels,totalPred))  # Need to feed it yTest not yTest_OneHot here
    print ("Overall Test Results: Av. precision {} +/- {}, Av. recall {}+/{},".format(pMn,pSd,rMn,rSd))
    #print(metrics.confusion_matrix(totalLabels, totalPred, sample_weight=None))
    
    #saver = tf.train.Saver()
    #saver.save(sess,"./3_Data/saves/model.ckpt")
    
    return

In [14]:
def SelectTestUsers(newUsers=10):
    newUsers = 10   # Num of randomly selected users to separate out
    con = sqlite3.connect(dbPath)

    # First reset back to 0
    con.execute("Update tblUsers Set TestUser = 0")
    con.commit()

    # Select random users
    sqlStr= "SELECT UserID FROM tblUsers Group by UserID ORDER BY RANDOM() LIMIT {}".format(newUsers)

    newUsersList = pd.read_sql_query(sqlStr, con)
    for row in newUsersList.itertuples():
        sqlStr = "Update tblUsers Set TestUser = 1 where UserID = {}".format(row[1])
        con.execute(sqlStr)
    con.commit()
    con.close()

    np.array(newUsersList).reshape(1,10)

<h3 style="background-color:green;color:white">4. ...And action!</h3>

In [ ]:
###### Model setup
loadFromSave = False
user_iteration = 30      # How many iterations of user selection
samplesPerUser = 2      # How many mini-batches to select for each user
batchRows = 50           # How many random periods to select in each mini-batch (=batch size)
batch_Iterations = 30   # How many iterations to perform on one ach mini-batch

n_steps = 336         # How many time steps (i.e. depth) to have
learning_rate = 0.05
n_hidden = 40 # hidden layer num of features
n_layers = 2
class1Weighting = 12
n_classes = 1  # 2 for one-hot

cellType = "LSTMCell"  # Choose: TimeFreqLSTMCell BasicLSTMCell
fieldList="UserID, t"
#fieldList="UserID, t, t1"
featureLen = len(fieldList.split(","))-2 # -2 as we drop UserID and t
tblName='tblTimeSeriesData'

ResetModel()
initializeModel(n_steps,featureLen,class1Weighting,loadFromSave)

print('\n Features len: {} User Iteration {} Samples per user: {} Batch iterations: {} Batch size {} N_steps {} Units {} Layers {} Weighting {}\n'.format(featureLen,user_iteration, samplesPerUser, batch_Iterations, batchRows, n_steps, n_hidden, n_layers, class1Weighting))
TrainModel(printOnBatchIteration=False,printOnSampleIteration=100)
TestModel(50)

